In [64]:
from typing import List, Dict, Any, Optional, Tuple
import requests
from thx_helper import process_stock_data_all,extract_json_from_js
from thx import BASE_URL
import logging
import pandas as pd
import talib
logger = logging.getLogger(__name__)

HEADERS = {
    'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/136.0.0.0 Safari/537.36',
    'X-Requested-With': 'XMLHttpRequest'
}
def _make_request(url: str, timeout: int = 10, headers: Dict[str, str] = {},**argv) -> Optional[Dict]:
    """发送HTTP请求并提取JSON数据"""
    try:
        print(url)
        common_header = HEADERS.copy()
        if headers:
            common_header.update(headers)
        response = requests.get(url, headers=common_header, timeout=timeout,**argv)
        response.raise_for_status()
        return response
    except requests.RequestException as e:
        logger.error(f"请求失败 {url}: {e}")
        return None
            
def history(code):
    url = f"{BASE_URL}/v6/line/{code}/01/all.js"
    print(url)
    respnse = _make_request(url)
    data = extract_json_from_js(respnse.text)
        
    if data:
        return process_stock_data_all(data)
    else:
        return []

code = 'hk_HK2018'
data = history(code)
df = pd.DataFrame(data)
print(df.columns)
df = df.rename(columns={'t':'date','o':'open','c':'close','h':'hight','l':'low','v':'volume',})
df['date'] = pd.to_datetime(df['date'], format='%Y%m%d')
# df.set_index('date', inplace = True)

https://d.10jqka.com.cn/v6/line/hk_HK2018/01/all.js
https://d.10jqka.com.cn/v6/line/hk_HK2018/01/all.js
Index(['t', 'v', 'o', 'c', 'h', 'l'], dtype='object')


In [65]:
df.tail(5)

,date,volume,open,close,hight,low
4918,2025-07-24,7696923,39.6,40.00,40.65,39.2
4919,2025-07-25,4417336,39.9,40.20,40.30,39.6
4920,2025-07-28,5340214,40.2,40.85,41.35,40.2
4921,2025-07-29,8658765,40.8,42.40,42.50,40.2
4922,2025-07-30,0,42.5,42.50,42.50,42.5


## RSI指标
https://www.earnmi.com/talib/momentum_indicators/RSI.html

RSI指标是相对强弱指标，全称为Relative Strength Index。它是一种用于衡量市场超买超卖状态的技术指标，由J. Welles Wilder于1978年提出。RSI指标的计算基于一定时期内的价格变动幅度，通过将一定时期内的上涨幅度和下跌幅度进行比较，以确定市场的强弱程度。

指标作用：
判断市场的超买超卖情况：当RSI指标超过70时，表示市场处于超买状态，可能出现回调或反转的机会；当RSI指标低于30时，表示市场处于超卖状态，可能出现反弹的机会。
确认趋势的延续或转变：当市场处于上升趋势时，RSI指标持续在50以上，表示趋势可能延续；当市场处于下降趋势时，RSI指标持续在50以下，表示趋势可能延续。如果RSI指标出现背离，即与价格走势相反，可能预示着趋势的转变。

确定买入卖出信号：当RSI指标从下方向上突破30时，可视为买入信号；当RSI指标从上方向下突破70时，可视为卖出信号。
指标公式
```
RSI = 100 - (100 / (1 + RS))
```
其中，RS表示相对强度，其计算公式为：
RS = (n日内平均上涨幅度的总和) / (n日内平均下跌幅度的总和)，一般来说，RSI指标的计算周期常用14日。

In [28]:
df['RSI'] = talib.RSI(df.c, timeperiod=14)
df.tail(5)

,t,v,o,c,h,l,RSI
4918,20250724,7696923,39.6,40.00,40.65,39.2,50.028566
4919,20250725,4417336,39.9,40.20,40.30,39.6,50.970884
4920,20250728,5340214,40.2,40.85,41.35,40.2,54.006456
4921,20250729,8658765,40.8,42.40,42.50,40.2,60.316089
4922,20250730,0,42.5,42.50,42.50,42.5,60.690764


## MA指标
Moving average 移动平均线,简称MA，原本的意思是移动平均，由于我们将其制作成线形，所以一般称之为移动平均线，简称均线。它是将某一段时间的收盘价之和除以该周期。 比如日线MA5指5天内的收盘价除以5 。

In [30]:
df['MA5'] = talib.MA(df.c,  timeperiod=5, matype=0)
df['MA20'] = talib.MA(df.c,  timeperiod=20, matype=0)
df.tail(5)

,t,v,o,c,h,l,RSI,MA5,MA20
4918,20250724,7696923,39.6,40.00,40.65,39.2,50.028566,39.82,40.3075
4919,20250725,4417336,39.9,40.20,40.30,39.6,50.970884,39.70,40.3025
4920,20250728,5340214,40.2,40.85,41.35,40.2,54.006456,39.89,40.3050
4921,20250729,8658765,40.8,42.40,42.50,40.2,60.316089,40.56,40.3900
4922,20250730,0,42.5,42.50,42.50,42.5,60.690764,41.19,40.5200


## EMA指标
EMA指标是指数移动平均线，全称为Exponential Moving Average。它是一种常用的技术分析工具，用于平滑价格数据并识别趋势。

指标作用
识别趋势：当价格位于EMA线之上时，表明市场处于上升趋势；当价格位于EMA线之下时，表明市场处于下降趋势。
交叉信号：当短期EMA线（如5日EMA）上穿长期EMA线（如20日EMA）时，被视为买入信号；当短期EMA线下穿长期EMA线时，被视为卖出信号。

指标公式
EMA（n）=（2 / (n + 1)） * 当前收盘价 +（1 - 2 / (n + 1)） * 上一期EMA（n）
其中，n为计算EMA的周期，当前收盘价为当日的收盘价。

In [31]:
df['EMA5']  = talib.EMA(df.c,  timeperiod=5)
df['EMA20']  = talib.EMA(df.c,  timeperiod=20)
df.tail(5)

,t,v,o,c,h,l,RSI,MA5,MA20,EMA5,EMA20
4918,20250724,7696923,39.6,40.00,40.65,39.2,50.028566,39.82,40.3075,39.907431,40.104556
4919,20250725,4417336,39.9,40.20,40.30,39.6,50.970884,39.70,40.3025,40.004954,40.113646
4920,20250728,5340214,40.2,40.85,41.35,40.2,54.006456,39.89,40.3050,40.286636,40.183775
4921,20250729,8658765,40.8,42.40,42.50,40.2,60.316089,40.56,40.3900,40.991091,40.394844
4922,20250730,0,42.5,42.50,42.50,42.5,60.690764,41.19,40.5200,41.494061,40.595335


## MACD指标
MACD指标的中文名称为移动平均线收敛/发散指标，英文名称为Moving Average Convergence Divergence。

MACD指标是一种趋势跟踪指标，用于衡量价格的快速移动平均线与慢速移动平均线之间的差异。它由两条线组成，一条为快速移动平均线（一般为12日EMA），另一条为慢速移动平均线（一般为26日EMA）。此外，还有一条被称为信号线的移动平均线（一般为9日EMA），用于产生买入和卖出信号。

指标作用

MACD指标主要用于判断价格的趋势和转折点。当快速移动平均线向上穿越慢速移动平均线时，产生买入信号；当快速移动平均线向下穿越慢速移动平均线时，产生卖出信号。同时，通过观察MACD线和信号线的交叉情况，可以进一步确认买卖信号。

指标公式

MACD线 = 快速移动平均线 - 慢速移动平均线 信号线 = MACD线的9日移动平均线

In [33]:
df['DIF'],df['DEM'],df['HISTOGRAM'] = talib.MACD(df.c,fastperiod=12, slowperiod=26, signalperiod=9)
df.tail(5)

,t,v,o,c,h,l,RSI,MA5,MA20,EMA5,EMA20,DIF,DEM,HISTOGRAM
4918,20250724,7696923,39.6,40.00,40.65,39.2,50.028566,39.82,40.3075,39.907431,40.104556,0.138117,0.318179,-0.180062
4919,20250725,4417336,39.9,40.20,40.30,39.6,50.970884,39.70,40.3025,40.004954,40.113646,0.132040,0.280951,-0.148911
4920,20250728,5340214,40.2,40.85,41.35,40.2,54.006456,39.89,40.3050,40.286636,40.183775,0.177625,0.260286,-0.082660
4921,20250729,8658765,40.8,42.40,42.50,40.2,60.316089,40.56,40.3900,40.991091,40.394844,0.334963,0.275221,0.059742
4922,20250730,0,42.5,42.50,42.50,42.5,60.690764,41.19,40.5200,41.494061,40.595335,0.462393,0.312656,0.149738


## BBANDS指标（布林带）
BBANDS称为布林带，英文名称为Bollinger Bands。

布林带是一种基于统计学原理的技术分析指标，由约翰·布林格（John Bollinger）于20世纪80年代提出。它通过计算价格的标准差来确定价格的高低波动区间，并以此构建出上下两条通道线，从而帮助判断价格的超买和超卖情况。

指标作用

判断价格的超买和超卖水平： + 当价格接近或触及上轨时，说明市场处于超买状态，可能会出现价格下跌的趋势； + 当价格接近或触及下轨时，说明市场处于超卖状态，可能会出现价格上涨的趋势。

识别价格的高低波动区间：布林带的宽度可以反映价格的波动性，当布林带收缩时，说明市场波动性较低；当布林带扩展时，说明市场波动性较高。

指标公式

计算中轨（MB），中轨是以一段时间的移动平均线为基础，常用的移动平均线周期为20天：

MB = MA(Close, N)

计算上轨（UP）和下轨（DN）：上轨和下轨是以中轨为基础，分别加上和减去一定比例的标准差。

UP = MB + K * σ DN = MB - K * σ

其中，K为调整因子，通常取值为2；σ为收盘价的标准差，计算公式为：

σ = StdDev(Close, N)

其中，Close为收盘价，MA为移动平均线，StdDev为标准差，N为计算周期。

In [35]:
df['UPPER'], df['MIDDLE'], df['LOWER'] = talib.BBANDS(df.c, timeperiod=20, nbdevup=2, nbdevdn=2, matype=1)
df.tail(5)

,t,v,o,c,h,l,RSI,MA5,MA20,EMA5,EMA20,DIF,DEM,HISTOGRAM,UPPER,MIDDLE,LOWER
4918,20250724,7696923,39.6,40.00,40.65,39.2,50.028566,39.82,40.3075,39.907431,40.104556,0.138117,0.318179,-0.180062,41.780048,40.104556,38.429063
4919,20250725,4417336,39.9,40.20,40.30,39.6,50.970884,39.70,40.3025,40.004954,40.113646,0.132040,0.280951,-0.148911,41.789795,40.113646,38.437497
4920,20250728,5340214,40.2,40.85,41.35,40.2,54.006456,39.89,40.3050,40.286636,40.183775,0.177625,0.260286,-0.082660,41.863030,40.183775,38.504519
4921,20250729,8658765,40.8,42.40,42.50,40.2,60.316089,40.56,40.3900,40.991091,40.394844,0.334963,0.275221,0.059742,42.302093,40.394844,38.487594
4922,20250730,0,42.5,42.50,42.50,42.5,60.690764,41.19,40.5200,41.494061,40.595335,0.462393,0.312656,0.149738,42.695906,40.595335,38.494763


## SAR指标

SAR称为抛物线指标，英文名称为Parabolic SAR (Stop and Reverse)。 抛物线指标是一种技术分析工具，用于确定股票或其他金融资产的趋势反转点。它的主要目的是提供交易者一个在何时停止损失和反转头寸的信号。

指标作用

抛物线指标可以帮助交易者确定交易的入场和出场点，以及设置适当的止损点。它在趋势市场中表现良好，并且能够捕捉到较长期的趋势。

指标公式

计算首个SAR值：

SAR1 = EP - (EP - SAR0) * AF

计算后续SAR值：

SARt = SARt-1 + (EP - SARt-1) * AF

其中，EP代表极点（Extreme Point），SAR0代表首个SAR值，SARt-1代表前一个周期的SAR值，SARt代表当前周期的SAR值，AF代表加速因子（Acceleration Factor）。

注意：SAR指标的计算需要一定的历史数据。在计算过程中，需要确定每个周期的最高价和最低价，并根据这些价格计算极点。在初始阶段，需要选择一个适当的SAR0值和AF值。

In [38]:
df['SAR'] = talib.SAR(df.h, df.l, acceleration=0.02, maximum=0.2)
df.tail(5)

,t,v,o,c,h,l,RSI,MA5,MA20,EMA5,EMA20,DIF,DEM,HISTOGRAM,UPPER,MIDDLE,LOWER,SAR
4918,20250724,7696923,39.6,40.00,40.65,39.2,50.028566,39.82,40.3075,39.907431,40.104556,0.138117,0.318179,-0.180062,41.780048,40.104556,38.429063,42.22400
4919,20250725,4417336,39.9,40.20,40.30,39.6,50.970884,39.70,40.3025,40.004954,40.113646,0.132040,0.280951,-0.148911,41.789795,40.113646,38.437497,42.14952
4920,20250728,5340214,40.2,40.85,41.35,40.2,54.006456,39.89,40.3050,40.286636,40.183775,0.177625,0.260286,-0.082660,41.863030,40.183775,38.504519,42.07653
4921,20250729,8658765,40.8,42.40,42.50,40.2,60.316089,40.56,40.3900,40.991091,40.394844,0.334963,0.275221,0.059742,42.302093,40.394844,38.487594,38.50000
4922,20250730,0,42.5,42.50,42.50,42.5,60.690764,41.19,40.5200,41.494061,40.595335,0.462393,0.312656,0.149738,42.695906,40.595335,38.494763,38.58000


## MOM指标
MOM金融指标的中文名称为动量指标，英文名称为Momentum Indicator。该指标是通过比较当前价格与一定期间前的价格变动情况来衡量市场的趋势力量。

指标作用

动量指标可以用于判断市场的超买和超卖情况，以及市场的趋势是否正在改变。当动量指标处于高位时，意味着市场可能已经过热，投资者应谨慎操作；而当动量指标处于低位时，意味着市场可能已经过冷，投资者可以寻找买入机会。

指标公式

动量指标的计算公式为：

MOM = 当前价格 - N期前的价格

其中，N代表所选取的期间，可以根据实际需求进行调整。一般来说，常用的期间为12个交易日或者26个交易日。



In [41]:
df['MOM'] = talib.MOM(df.c, timeperiod=12)
df.tail(5)

,t,v,o,c,h,l,RSI,MA5,MA20,EMA5,EMA20,DIF,DEM,HISTOGRAM,UPPER,MIDDLE,LOWER,SAR,MOM12,MOM
4918,20250724,7696923,39.6,40.00,40.65,39.2,50.028566,39.82,40.3075,39.907431,40.104556,0.138117,0.318179,-0.180062,41.780048,40.104556,38.429063,42.22400,-0.40,-0.40
4919,20250725,4417336,39.9,40.20,40.30,39.6,50.970884,39.70,40.3025,40.004954,40.113646,0.132040,0.280951,-0.148911,41.789795,40.113646,38.437497,42.14952,0.45,0.45
4920,20250728,5340214,40.2,40.85,41.35,40.2,54.006456,39.89,40.3050,40.286636,40.183775,0.177625,0.260286,-0.082660,41.863030,40.183775,38.504519,42.07653,-0.20,-0.20
4921,20250729,8658765,40.8,42.40,42.50,40.2,60.316089,40.56,40.3900,40.991091,40.394844,0.334963,0.275221,0.059742,42.302093,40.394844,38.487594,38.50000,2.40,2.40
4922,20250730,0,42.5,42.50,42.50,42.5,60.690764,41.19,40.5200,41.494061,40.595335,0.462393,0.312656,0.149738,42.695906,40.595335,38.494763,38.58000,3.20,3.20


## OBV指标
OBV指标（On-Balance Volume）能量潮由Joe Granville提出，通过统计成交量变动的趋势推测股价趋势，用于衡量成交量的变化趋势和预测价格趋势的强弱。OBV指标通过统计成交量的正负值来判断市场的买卖力量，从而预测价格的上涨或下跌趋势。

指标作用

判断市场的买卖力量：当OBV指标上升时，表示买盘力量较强，市场可能会进一步上涨；当OBV指标下降时，表示卖盘力量较强，市场可能会进一步下跌。

预测价格趋势的强弱：当市场价格上涨时，OBV指标也在上升，说明买盘力量较强，价格上涨可能会持续；当市场价格下跌时，OBV指标也在下降，说明卖盘力量较强，价格下跌可能会持续。

以“N”字型为波动单位，一浪高于一浪称“上升潮”，下跌称“跌潮”；上升潮买进，跌潮卖出

须配合K线图走势

用多空比率净额法进行修正，但不知TA-Lib采用哪种方法

确认价格趋势的可靠性：当市场价格出现突破时，若OBV指标也出现突破，可以确认价格趋势的可靠性。

需要注意的是，OBV指标是一种相对较简单的量能指标，仅通过成交量来判断市场买卖力量的强弱，因此在使用时需要结合其他技术指标和市场行情进行综合分析。

指标公式

OBV = OBV前一日 + 当日的成交量 × 成交量的正负值

其中，成交量的正负值的计算方式如下： + 如果当日收盘价大于前一日收盘价，成交量的正负值为当日成交量； + 如果当日收盘价小于前一日收盘价，成交量的正负值为当日成交量的负值； + 如果当日收盘价等于前一日收盘价，成交量的正负值为0。

On Balance Volume 能量潮由Joe Granville提出，通过统计成交量变动的趋势推测股价趋势

计算公式

以某日为基期，逐日累计每日上市股票总成交量，若隔日指数或股票上涨 ，则基期OBV加上本日成交量为本日OBV。隔日指数或股票下跌， 则基期OBV减去本日成交量为本日OBV.

多空比率净额= [（收盘价－最低价）－（最高价-收盘价）] ÷（ 最高价－最低价）×成交量

In [42]:
df['OBV'] = talib.OBV(df.c, df.v)
df.tail(5)

,t,v,o,c,h,l,RSI,MA5,MA20,EMA5,...,DIF,DEM,HISTOGRAM,UPPER,MIDDLE,LOWER,SAR,MOM12,MOM,OBV
4918,20250724,7696923,39.6,40.00,40.65,39.2,50.028566,39.82,40.3075,39.907431,...,0.138117,0.318179,-0.180062,41.780048,40.104556,38.429063,42.22400,-0.40,-0.40,565671995.0
4919,20250725,4417336,39.9,40.20,40.30,39.6,50.970884,39.70,40.3025,40.004954,...,0.132040,0.280951,-0.148911,41.789795,40.113646,38.437497,42.14952,0.45,0.45,570089331.0
4920,20250728,5340214,40.2,40.85,41.35,40.2,54.006456,39.89,40.3050,40.286636,...,0.177625,0.260286,-0.082660,41.863030,40.183775,38.504519,42.07653,-0.20,-0.20,575429545.0
4921,20250729,8658765,40.8,42.40,42.50,40.2,60.316089,40.56,40.3900,40.991091,...,0.334963,0.275221,0.059742,42.302093,40.394844,38.487594,38.50000,2.40,2.40,584088310.0
4922,20250730,0,42.5,42.50,42.50,42.5,60.690764,41.19,40.5200,41.494061,...,0.462393,0.312656,0.149738,42.695906,40.595335,38.494763,38.58000,3.20,3.20,584088310.0


## ATR指标

Average True Range 真实波动幅度均值（ATR)是 以 N 天的指数移动平均数平均後的交易波动幅度。 计算公式：一天的交易幅度只是单纯地 最大值 - 最小值。

简单来说ATR指标是对 TR指标 的平滑处理

指标作用

ATR指标可以帮助交易者判断市场的波动性水平，从而确定合适的止损点位和目标价格。它也可以用于确定市场的趋势，例如当ATR值上升时，表示市场波动加大，可能出现较大的行情变动。

指标公式

计算真实波动幅度（TR）：

TR = Max(H - L, H - Cp, Cp - L)

其中H表示当日的最高价，L表示当日的最低价，Cp表示前一日的收盘价。

计算平均真实波动幅度（ATR）：

ATR = (ATRn-1 * (n-1) + TR) / n

其中ATRn-1表示前一日的ATR值，n表示计算ATR的周期长度。一般来说，ATR的周期长度为14天。


In [43]:
df['ATR'] = talib.ATR(df.h, df.l, df.c,timeperiod=14)
df.tail(5)

,t,v,o,c,h,l,RSI,MA5,MA20,EMA5,...,DEM,HISTOGRAM,UPPER,MIDDLE,LOWER,SAR,MOM12,MOM,OBV,ATR
4918,20250724,7696923,39.6,40.00,40.65,39.2,50.028566,39.82,40.3075,39.907431,...,0.318179,-0.180062,41.780048,40.104556,38.429063,42.22400,-0.40,-0.40,565671995.0,1.661524
4919,20250725,4417336,39.9,40.20,40.30,39.6,50.970884,39.70,40.3025,40.004954,...,0.280951,-0.148911,41.789795,40.113646,38.437497,42.14952,0.45,0.45,570089331.0,1.592844
4920,20250728,5340214,40.2,40.85,41.35,40.2,54.006456,39.89,40.3050,40.286636,...,0.260286,-0.082660,41.863030,40.183775,38.504519,42.07653,-0.20,-0.20,575429545.0,1.561212
4921,20250729,8658765,40.8,42.40,42.50,40.2,60.316089,40.56,40.3900,40.991091,...,0.275221,0.059742,42.302093,40.394844,38.487594,38.50000,2.40,2.40,584088310.0,1.613983
4922,20250730,0,42.5,42.50,42.50,42.5,60.690764,41.19,40.5200,41.494061,...,0.312656,0.149738,42.695906,40.595335,38.494763,38.58000,3.20,3.20,584088310.0,1.505841
